# Regularized Linear Models

Overfitting을 줄이는 좋은 방법은 model을 regularize하는 것이다. Polynomial model에 대해서는 간단히 polynomial degree를 줄이는 방법이 있다.

Linear model에 대한 일반적인 regularization은 model의 weight를 제한하는 것이다. 여기서는 weight를 제한하는 3가지 방법인 Ridge Regression, Lasso Regression, Elastic Net에 대해 알아본다.

## Ridge Regression

**Tikhonov regularization**라고도 부르는 **Ridge Regression**은 cost function에 **regularization term** $\alpha \sum^{n}_{i=1} \theta_i^2$를 추가하는 방법이다. 이는 알고리즘이 data에 fit하면서 가능한 작은 model weight를 가지게 한다.

hyperparameter $\alpha$는 regularization의 정도를 조절한다. $\alpha=0$이면 Ridge Regression은 Linear Regression이 되고, $\alpha$가 매우 크면, 모든 weight는 0에 가까워지며, 그 결과로 data의 평균을 지나는 수평선의 형태가 된다. 다음은 Ridge Regression의 cost function 식이다.

**Ridge Regression cost function**

$J(\mathbf{\theta}) = MSE(\mathbf{\theta}) + \alpha \dfrac{1}{2} \sum^n_{i=1} \theta^2_i$

bias term인 $\theta_0$는 regularized되지 않는다. 만약, feature의 weight들($\theta_1$~$\theta_n$)의 vector를 $\mathbf{w}$라고 정의하면, regularization term은 $\frac{1}{2}(\Vert{\mathbf{w}}\Vert_2)^2$로 간단히 나타낼 수 있다. 여기서, $\Vert{\mathbf{w}}\Vert_2$는 weight vector의 $l_2$ norm을 나타낸다. Gradient Descent에서는, 간단히 $\alpha \mathbf{w}$을 MSE gradient vector에 추가해주면 된다.

아래 그림은 여러 $\alpha$값으로 linear data에 대한 Ridge model을 학습시킨 결과를 나타낸 것이다.

<br/>
<img src="./images/ridge_regression.png" alt="ridge_regression"> 
<br/>

왼쪽은 plain Ridge model을 사용하였고, 오른쪽은 `PolynomialFeatures(degree=10)`으로 확장하고 `StandardScaler`로 scaling을 해준 feature에 대해 Ridge model을 적용한 것이다. 따라서, 오른쪽의 경우는 Ridge Regularization을 적용한 Polynomial Regression이 된다. 여기서 $\alpha$가 커질수록, flat해지는 것을 볼 수 있는데, 이는 variance가 감소하고, bias가 증가한다는 것을 의미한다.

Ridge Regression도 Linear Regression처럼 closed-form equation(normal equation)이나 Gradient Descent에 적용할 수 있다.

다음은 closed-form solution의 식이다. (여기서 $\mathbf{A}$는 bias term의 위치인 맨 왼쪽 위의 원소가 0인 (n+1)X(n+1) 크기의 identity matrix이다.)

**Ridge Regression closed-form solution**

$\hat{\mathbf{\theta}} = \big( \mathbf{X}^T \mathbf{X} + \alpha \mathbf{A} \big)^{-1} \mathbf{X}^T \mathbf{y}$ (Normal Equation식에서 $\alpha \mathbf{A}$만 추가된 형태)

다음은 위 식을 사이킷런에서 수행하는 코드이다.

In [1]:
import numpy as np
from sklearn.linear_model import Ridge

m = 20
X = 3 * np.random.rand(m, 1)
y = 1 + 0.5 * X + np.random.randn(m, 1) / 1.5

ridge_reg = Ridge(alpha=1, solver='cholesky')
ridge_reg.fit(X, y)
ridge_reg.predict([[1.5]])

array([[1.65168174]])

Stochasic Gradient Descent에서 수행하면 다음과 같다.

In [2]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(penalty="l2")
sgd_reg.fit(X, y.ravel())
sgd_reg.predict([[1.5]])

array([1.50652898])

SGD에서의 Ridge Regression은 `penalty` hyperparameter를 통해 $l_2$ norm을 추가해주면 된다.

## Lasso Regression

**Lasso Regression(Least Absolute Shrinkage and Selection Operator Regression)**은 Ridge Regression처럼 cost function에 regularization term을 추가하는 방법이지만 weight vector의 $l_2$ norm이 아닌 $l_1$ norm을 추가하는 방법이다.

**Lasso Regression cost function**

$J(\mathbf{\theta}) - MSE(\mathbf{\theta}) + \alpha \sum^n_{i=1} \vert\theta_i\vert$

아래 그림은 여러 $\alpha$값으로 linear data에 대한 Lasso model을 학습시킨 결과를 나타낸 것이다.

<br/>
<img src="./images/lasso_regression.png" alt="lasso_regression"> 
<br/>

Lasso Regression의 중요한 특징은 가장 덜 중요한 feature의 weight를 완전히 제거하려고(0으로 만드려고) 한다는 것이다. 예를 들어, 오른쪽 그래프의 dash line($\alpha=1e-7$)은 quadratic처럼 보이지만, 거의 linear한데, 이는 모든 high-degree polynomial feature의 weight를 0으로 만들었기 때문이다.

Lasso의 cost function은 $\theta_i = 0$ ($i=1, 2, \cdots, n$)이 되는 경우, 미분이 불가능해지지만, **subgradient vector** $\mathbf{g}$를 0이 된 $\theta_i$ 대신에 사용하면 Gradient Descent를 계속해서 수행할 수 있다.

**Lasso Regression subgradient vector**

$g(\mathbf{\theta}, J) = \nabla_{\mathbf{\theta}}MSE(\mathbf{\theta}) + \alpha \begin{pmatrix} \text{sign}(\theta_1) \\ \text{sign}(\theta_2) \\ \vdots \\ \text{sign}(\theta_n) \end{pmatrix}$, $\text{ where sign}(\theta_i) = \begin{equation} \begin{cases} -1 & \text{if} & \theta_i \lt 0 \\ 0 & \text{if} & \theta_i = 0 \\ +1 & \text{if} & \theta_i \gt 0 \end{cases} \end{equation}$

다음은 사이킷런에서 Lasso Regression을 사용하는 예이다.

In [3]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X, y)
print(lasso_reg.predict([[1.5]]))

[1.66746483]


다음과 같이 `SGDRegressor`에 l1 norm을 설정하는 방법도 있다.

In [4]:
sgd_lasso_reg = SGDRegressor(penalty="l1")
sgd_lasso_reg.fit(X, y.ravel())
print(sgd_lasso_reg.predict([[1.5]]))

[1.50148063]


## Elastic Net

**Elastic Net**도 위의 두 방법과 마찬가지로 regularization term을 추가하는 방법이다. Elastic Net의 regularization term은 Ridge와 Lasso의 term을 섞은 형태이며 섞이는 비율은 $r$값을 통해 조절할 수 있다.

다음의 식은 Elastic Net의 cost function이며, $r=0$인 경우, Ridge Regression이 되며, $r=1$인 경우, Lasso Regression이 된다는 것을 알 수 있다.

**Elastic Net cost function**

$J(\mathbf{\theta}) = MSE(\mathbf{\theta}) + r\alpha\sum^n_{i=1} \vert\theta_i\vert + \dfrac{1-r}{2} \alpha \sum^n_{i=1} \theta^2_i$

그렇다면 regularization이 없는 plain Linear Regression, Ridge, Lasso, Elastic Net중에서 어떤 모델을 사용해야 할까? 대부분의 경우에서 약간의 regularization은 필요하므로, Linear Regression은 잘 사용하지 않게 될 것이다. Ridge는 default로 사용하기 좋으며 만약 몇개의 feature가 유용하지 않게 느껴진다면, Lasso나 Elastic Net을 사용해 유용하지 않은 feature의 weight를 줄이거나 0으로 만들면 된다. 일반적으로 Elastic Net이 Lasso보다는 선호되는데, 그 이유는 feature의 수가 instance의 수보다 많을 경우나 몇개의 feature들이 매우 correlated한 경우에는 Lasso가 잘 동작하지 않기 때문이다.

Elastic Net은 다음과 같이 사이킷런의 `ElasticNet`을 사용하면 된다.

In [5]:
from sklearn.linear_model import ElasticNet

elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic_net.fit(X, y)
elastic_net.predict([[1.5]])

array([1.66076306])

## Early Stopping

**Early Stopping**이란 validation error가 minimum에 도달하면 training을 멈추는 방법이다.

아래의 그림은 Batch Gradient Descent로 complex model(high-degree Polynomial Regression model)을 학습하는 것이다. epoch이 커지며 training과 validation의 error(RMSE)가 줄어들지만 어느시점부터는 validation만 다시 커지게된다. 이때가 model의 overfitting이 시작되는 지점이다. early stopping에서는, 이 지점(validation error가 minimum이 되는 지점)에서 학습을 멈춘다.

<br/>
<img src="./images/early_stopping.png" alt="early_stopping"> 
<br/>

Stochastic과 Mini-batch Gradient Descent의 경우, 위의 그래프처럼 curve가 매끄럽지 못하므로 minimum에 도달한 것인지 판단하기 어렵다. 한가지 해결책은 여러번의 minimum이 일어난 후에 학습을 멈추는 것이다. 그리고 validation error가 minimum이었던 parameter로 돌아가는 것이다.

다음의 코드는 early stopping을 구현한 것이다.

In [ ]:
from sklearn.base import clone

poly_scaler = Pipeline([
    ("poly_features", PolynomialFeatures(degree=90, include_bias=False)),
    ("std_scaler", StandardScaler())
])
X_train_poly_scaled = poly_scaler.fit_transform(X_train)
X_val_poly_sclaed = poly_scaler.transform(X_val)

# warm_start를 설정하면,
# fit()을 호출할 때마다 처음부터 새로 학습시키는 것이 아니라
# 이전에 학습한 결과에서 이어서 학습하게됨
sgd_reg = SGDRegressor(max_iter=1, tol=-np.infty, warm_start=True,
                       penalty=None, learning_rate='constant', eta0=0.0005)

minimum_val_error = float("inf")
best_epoch = None
best_model = None
for epoch in range(1000):
    sgd_reg.fit(X_train_poly_scaled, y_train)
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)
    val_error = mean_squared_error(y_val, y_val_predict)
    if val_error < minimum_val_error:
        minimum_val_error = val_error
        best_epoch = epoch
        best_model = clone(sgd_reg)

위 코드에서처럼, `warm_start=True`를 설정하면, 이전에 학습된 모델에서 이어서 학습할 수 있으므로, early stopping을 구현할 수 있다.